# 1. Most downloaded otome games in 2025 Q1, data from [Sensortower](sensortower.com[https://sensortower.com/blog/2025-q1-unified-top-5-otome%20games-units-us-60789073241bc16eb88c6e7f])

**1** [Love and deepspace](https://app.sensortower.com/overview/654319170b276e064b329296?country=US)  
**2** [Dorian: Interactive Dramas Hub](https://app.sensortower.com/overview/60038cf5c86cbe76cd846cfa?country=US)  
**3** [Boba Story](https://app.sensortower.com/overview/610f77279c96214ec4fe1388?country=US)  
**4** [Anime Dating Sim: Steamy Waifu](https://app.sensortower.com/overview/647533dd5d2a176ec9b3b678?country=US)  
**5** [NU: Carnival - Bliss](https://app.sensortower.com/overview/6582614bb759a80dbc1cbcc4?country=US)



# 2. Most loved Anime characters, results from [MyAnimeList](https://myanimelist.net/character.php)

In [4]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [144]:
base_url = "https://myanimelist.net/character.php?limit="
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) " +
                  "AppleWebKit/537.36 (KHTML, like Gecko) " +
                  "Chrome/58.0.3029.110 Safari/537.3"
}

all_characters = []

for offset in range(0, 1000, 50):
    page_url = f"{base_url}{offset}"
    response = requests.get(page_url, headers=headers)
    time.sleep(3)
    soup = BeautifulSoup(response.text, "html.parser")
    rows = soup.find_all("tr", class_="ranking-list")
    
    if not rows:
        print(f"⚠️ No characters found at offset {offset} — possible block or page layout change.")
        break
    
    for roles in rows:
        rank = roles.find("td", class_="rank").text.strip()
        name = roles.find("td", class_="people").text.strip()
        anime = roles.find("td", class_="animeography").text.strip()
        favorites = roles.find("td", class_="favorites").text.strip().replace(",", "")
        url = roles.find("a")["href"]
        all_characters.append({
            "rank": rank,
            "name": name,
            "anime": anime,
            "favorites": int(favorites),
            "url": url
        })
    print(f"{len(all_characters)} characters scraped")  # use len(all_characters)!
    time.sleep(3)

print(f"top {len(all_characters)} saved")


50 characters scraped
100 characters scraped
150 characters scraped
200 characters scraped
250 characters scraped
300 characters scraped
350 characters scraped
400 characters scraped
450 characters scraped
500 characters scraped
550 characters scraped
600 characters scraped
650 characters scraped
700 characters scraped
750 characters scraped
800 characters scraped
850 characters scraped
900 characters scraped
950 characters scraped
1000 characters scraped
top 1000 saved


In [146]:
df = data = pd.DataFrame(all_characters)
df.to_csv("top_1000_mal_characters_scraped.csv", index=False)

In [148]:
data = pd.read_csv("top_1000_mal_characters_scraped.csv")
data.head()

,rank,name,anime,favorites,url
0,1,"Lamperouge, Lelouch(ルルーシュ・ランペルージ)",Code Geass: Hangyaku no LelouchCode Geass: Han...,173552,https://myanimelist.net/character/417/Lelouch_...
1,2,"Monkey D., Luffy(モンキー・D・ルフィ)",One PieceOne Piece Movie 01One Piece Movie 02:...,143840,https://myanimelist.net/character/40/Luffy_Mon...
2,3,Levi(リヴァイ),Shingeki no Kyojin: Kuinaki SentakuShingeki no...,143364,https://myanimelist.net/character/45627/Levi
3,4,"Lawliet, L(エル ローライト)",Death NoteDeath Note: Rewrite,128375,https://myanimelist.net/character/71/L_Lawliet
4,5,"Roronoa, Zoro(ロロノア・ゾロ)",One PieceOne Piece Movie 01One Piece Movie 02:...,113243,https://myanimelist.net/character/62/Zoro_Roronoa


### scrape individual pages

In [150]:
from playwright.async_api import async_playwright
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless=False)
page = await browser.new_page()

In [151]:
import requests
from bs4 import BeautifulSoup
import nest_asyncio
import asyncio

async def extract_content(url):
    try:

        await page.goto(url)
        html = await page.content()
        soup = BeautifulSoup(html, 'html.parser')
        
        normal_header = soup.find('h2', class_='normal_header')
        if not normal_header:
            return "Normal header not found."
        
        content = []
        current = normal_header.next_sibling
        
        while current:
            if current.name == 'div' and 'spoiler' in current.get('class', []):
                break  # Stop at the first spoiler
            if isinstance(current, str):
                content.append(current.strip())
            elif current.name in ['br']:
                content.append('\n')  # Handle line breaks
            elif current.name:
                # Extract text from other tags (e.g., <b>, <a>)
                content.append(current.get_text(strip=True))
            current = current.next_sibling
        
        # Combine and clean the extracted text
        extracted_text = ' '.join(content).replace('\n\n', '\n').strip()
        return extracted_text
    
    except Exception as e:
        return f"Error processing {url}: {str(e)}"

In [152]:
urls = data['url']
descriptions = []
nest_asyncio.apply()
for url in urls:
    result = await extract_content(url)
    time.sleep(2)
    descriptions.append(result)

In [155]:
d = pd.DataFrame(descriptions, columns=['description'])
merged = pd.concat([data, d], axis=1)

In [116]:
info = {}

for line in result.split("\n"):
    line = line.strip()
    if line.startswith("Age:"):
        info["Age"] = line.replace("Age:", "").strip()
    elif line.startswith("Birthdate:"):
        info["Birthday"] = line.replace("Birthdate:", "").strip()
    elif line.startswith("Nationality:"):
        info["Nationality"] = line.replace("Nationality:", "").strip()
    elif line.startswith("Height:"):
        info["Height"] = line.replace("Height:", "").strip()
    elif line.startswith("Occupation:"):
        info["Occupation"] = line.replace("Occupation:", "").strip()

for key, value in info.items():
    print(f"{key}: {value}")

Age: 17 (first season), 18 (second season)
Birthday: December 5, 1999 a.t.b. (Sagittarius)
Nationality: Britannia
Height: 178 cm (5'10")
Occupation: Student, Terrorist, Knightmare Pilot, Politician, Emperor, Prince


In [173]:
def extract_info(text):
    info = {
        "Age": None,
        "Birthday": None,
        "Nationality": None,
        "Height": None,
        "Occupation": None
    }
    for line in text.split("\n"):
        line = line.strip()
        if line.startswith("Age:"):
            info["Age"] = line.replace("Age:", "").strip()
        elif line.startswith("Birthdate:"):
            info["Birthday"] = line.replace("Birthdate:", "").strip()
        elif line.startswith("Nationality:"):
            info["Nationality"] = line.replace("Nationality:", "").strip()
        elif line.startswith("Height:"):
            info["Height"] = line.replace("Height:", "").strip()
        elif line.startswith("Occupation:"):
            info["Occupation"] = line.replace("Occupation:", "").strip()
    return pd.Series(info)

# Apply the function row by row and expand to new columns
merged[["Age", "Birthday", "Nationality", "Height", "Occupation"]] = merged["description"].apply(extract_info)


In [ ]:
texts = merged['description']
for text in texts:
    

### Get the image

In [176]:
async def extract_image(url):
    try:

        await page.goto(url)
        html = await page.content()
        soup = BeautifulSoup(html, 'html.parser')
        
        # Find the image
        image = soup.find("div", id="content").find('td').find('img')['src']
        return image
    except Exception as e:
        return f"Error processing {url}: {str(e)}"

In [178]:
urls = data['url']
images = []
nest_asyncio.apply()
for url in urls:
    result = await extract_image(url)
    time.sleep(2)
    images.append(result)

In [177]:
url = 'https://myanimelist.net/character/2821/Ryousuke_Takahashi'
result = await extract_image(url)
result

'https://cdn.myanimelist.net/images/characters/12/296426.jpg'

In [179]:
images1 = pd.DataFrame(images, columns=['image'])
merged = pd.concat([merged, images1], axis=1)

In [180]:
merged.to_csv('merged.csv',index = False)

In [6]:
merged = pd.read_csv('merged.csv')
merged.head()

,rank,name,anime,favorites,url,description,image,Age,Birthday,Nationality,Height,Occupation,image.1
0,1,"Lamperouge, Lelouch(ルルーシュ・ランペルージ)",Code Geass: Hangyaku no LelouchCode Geass: Han...,173552,https://myanimelist.net/character/417/Lelouch_...,"Age: 17 (first season), 18 (second season) \n ...",NaN,"17 (first season), 18 (second season)","December 5, 1999 a.t.b. (Sagittarius)",Britannia,"178 cm (5'10"")","Student, Terrorist, Knightmare Pilot, Politici...",Error processing https://myanimelist.net/chara...
1,2,"Monkey D., Luffy(モンキー・D・ルフィ)",One PieceOne Piece Movie 01One Piece Movie 02:...,143840,https://myanimelist.net/character/40/Luffy_Mon...,"Age: 17; 19 \n Birthdate: May 5, Taurus \n Hei...",NaN,17; 19,"May 5, Taurus",NaN,172 cm; 174 cm,NaN,https://cdn.myanimelist.net/images/characters/...
2,3,Levi(リヴァイ),Shingeki no Kyojin: Kuinaki SentakuShingeki no...,143364,https://myanimelist.net/character/45627/Levi,"Birthday: December 25 \n Height: 160 cm (5'3"")...",NaN,NaN,NaN,NaN,"160 cm (5'3"")",NaN,https://cdn.myanimelist.net/images/characters/...
3,4,"Lawliet, L(エル ローライト)",Death NoteDeath Note: Rewrite,128375,https://myanimelist.net/character/71/L_Lawliet,"Birthday: October 31, 1979 (1982 in anime) \n ...",NaN,NaN,NaN,NaN,"179 cm (5'10"")",NaN,Error processing https://myanimelist.net/chara...
4,5,"Roronoa, Zoro(ロロノア・ゾロ)",One PieceOne Piece Movie 01One Piece Movie 02:...,113243,https://myanimelist.net/character/62/Zoro_Roronoa,Error processing https://myanimelist.net/chara...,NaN,NaN,NaN,NaN,NaN,NaN,https://cdn.myanimelist.net/images/characters/...


## Find the gender 

In [7]:
def detect_gender(desc):
    desc = desc.lower()
    female_keywords = ['she', 'her', 'woman', 'girl', 'female', 'lady']
    male_keywords = ['he', 'him', 'man', 'boy', 'male', 'gentleman']
    
    # Count matches
    female_score = sum(word in desc for word in female_keywords)
    male_score = sum(word in desc for word in male_keywords)

    if female_score > male_score:
        return 'female'
    elif male_score > female_score:
        return 'male'
    else:
        return 'unknown'

In [8]:
merged['gender'] = merged['description'].apply(detect_gender)

AttributeError: 'float' object has no attribute 'lower'

# 3. Use NLP Model to get characters ethnicity impression